In [2]:
import json
import random
import re
import pandas as pd
from datasets import Dataset, DatasetDict, load_from_disk
import os
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

# Arrow 파일 경로 설정
train_file_path = os.path.join(parent_dir, 'data', 'train_dataset')
test_file_path = os.path.join(parent_dir, 'data', 'test_dataset')
dataset = load_from_disk(train_file_path)
test_dataset=load_from_disk(test_file_path)

wiki_path = os.path.join(parent_dir, 'data', "wikipedia_documents.json")

with open(wiki_path, "r", encoding="utf-8") as json_file:
    json_data = json.load(json_file)


df_wiki=pd.read_json(wiki_path).transpose()

In [3]:
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
test_dataset=test_dataset

train_df=pd.DataFrame(train_dataset)
valid_df=pd.DataFrame(valid_dataset)
test_df=pd.DataFrame(test_dataset)

print(f"Train length : {len(train_df)}")
print(f"Valid length : {len(valid_df)}")
print(f"Test length : {len(test_df)}")
print(f"df_wiki length : {len(df_wiki)}")

Train length : 3952
Valid length : 240
Test length : 600
df_wiki length : 60613


In [6]:
train_df.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [27]:
def generate_synonyms(question):
    """
    질문을 받아서 동의어로 변형된 여러 질문을 생성하는 함수
    """
    
    client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)
    
    # 동의어 변형을 위한 간단한 예시 (직접 구축하거나, 동의어 사전 또는 paraphrasing 모델 사용 가능)
    synonyms = [question]
    try:
        # GPT 또는 다른 언어 모델을 사용한 동의어 질문 생성
        prompt = f"mrc 질문을 증강하는거야. 다음 질문에 대해 동의어로 바꾼 질문을 하나만 생성하고 질문만 출력해줘: '{question}'"
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",  # 사용할 모델
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt}],
            max_tokens=100,
            temperature=0.5,
        )
        # 응답 처리 부분: 생성된 질문 추출
        response_text = completion.choices[0].message.content.strip()
        return response_text

    except Exception as e:
        print(f"Error generating synonyms: {e}")
        return None

In [30]:
import pandas as pd
import os
import random
from openai import OpenAI

# 파일 경로 설정
csv_file_path = "mrc_augmented_questions_data.csv"

# 빈 리스트 생성: 저장할 데이터를 담기 위한 리스트
data = []

# 학습 데이터에서 질문 생성 및 동의어 질문 추가
total_questions = len(train_df)  # 전체 데이터 수 계산

for index, row in train_df.iterrows():
    
    # 기존 질문 추출
    question = row['question']
    
    if question:

        # 기존 질문을 데이터에 추가
        data.append({
            "title": row['title'],
            "context": row['context'],
            "document_id": row['document_id'],
            "question": question,
            "id": row["id"],
            "answers": row["answers"],
            "augmented_question": False
        })
        
        # 동의어로 변형된 질문 생성
        augmented_question = generate_synonyms(question)
        
        # 현재 몇 번째 항목을 처리 중인지 출력
        print(f"Processing question {index + 1}/{total_questions} : {question}_____________{augmented_question}")
        
        # 동의어 질문을 데이터에 추가
        data.append({
            "title": row['title'],
            "context": row['context'],
            "document_id": row['document_id'],
            "question": augmented_question,
            "id": row["id"],
            "answers": row["answers"],
            "augmented_question": True
        })

# 새로운 데이터를 데이터프레임으로 변환
df_new = pd.DataFrame(data)

# 데이터프레임을 CSV 파일로 저장 (덮어쓰기)
df_new.to_csv(csv_file_path, index=False)

# 데이터프레임 출력
print(df_new)


Processing question 1/3952 : 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?_____________미국의 행정부 견제권을 갖는 국가 기관은 무엇입니까?
Processing question 2/3952 : 현대적 인사조직관리의 시발점이 된 책은?_____________'현대적 인사관리의 원조서는 무엇인가요?'
Processing question 3/3952 : 강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?_____________강희제가 1717년에 작성한 글은 누구를 위해 쓰여졌습니까?
Processing question 4/3952 : 11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?_____________11~12세기에 제작된 본존불은 주로 어떤 국가의 특징이 전파되었나요?
Processing question 5/3952 : 명문이 적힌 유물을 구성하는 그릇의 총 개수는?_____________유물에 새겨진 문구를 이루는 그릇의 전체 수는 몇 개인가요?
Processing question 6/3952 : 카드모스의 부하들이 간 곳에는 무엇이 있었는가?_____________카드모스의 부하들이 방문한 장소에는 어떤 것이 있었습니까?
Processing question 7/3952 : 관우를 불태워 죽이려한 사람 누구인가?_____________누가 관우를 불태워 죽이려 했습니까?
Processing question 8/3952 : 참호 속에 무기와 장비를 버리고 도주한 집단은?_____________어떤 집단이 무기와 장비를 버리고 도망쳤습니까?
Processing question 9/3952 : 제2차 세계 대전에 참전하여 사망한 자식은?_____________누가 제2차 세계 대전에 참전하여 사망했습니까?
Processing question 10/3952 : 고려 공민왕이 처가 식구들과 아내와 함께 피신처로 삼은 마을은?_____________"공민왕이 가족들과 함께 피신한 마을은 어디였나요?

In [34]:
from datasets import load_from_disk
import os
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

# Arrow 파일 경로 설정
train_file_path = os.path.join(parent_dir, 'data', 'train_dataset')
test_file_path = os.path.join(parent_dir, 'data', 'test_dataset')
dataset = load_from_disk(train_file_path)
test_dataset=load_from_disk(test_file_path)

In [35]:
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
test_dataset=test_dataset['validation']

train_df=pd.DataFrame(train_dataset)
valid_df=pd.DataFrame(valid_dataset)
test_df=pd.DataFrame(test_dataset)

print(f"Train length : {len(train_df)}")
print(f"Valid length : {len(valid_df)}")
print(f"Test length : {len(test_df)}")

Train length : 3952
Valid length : 240
Test length : 600


In [36]:
train_df.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [41]:

# 어그멘테이션된 CSV 파일 로드
augmentation_csv_path = 'mrc_augmented_questions_data.csv'
augmented_df = pd.read_csv(augmentation_csv_path)

print(augmented_df.head())

# type str -> object(dict)로 변환
augmented_df["answers"] = augmented_df["answers"].apply(lambda x: eval(x))


    title                                            context  document_id  \
0   미국 상원  미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...        18293   
1   미국 상원  미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...        18293   
2  인사조직관리  '근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...        51638   
3  인사조직관리  '근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...        51638   
4     강희제  강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...         5028   

                           question            id  \
0  대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?  mrc-1-000067   
1     미국의 행정부 견제권을 갖는 국가 기관은 무엇입니까?  mrc-1-000067   
2            현대적 인사조직관리의 시발점이 된 책은?  mrc-0-004397   
3           '현대적 인사관리의 원조서는 무엇인가요?'  mrc-0-004397   
4    강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?  mrc-1-000362   

                                         answers  augmented_question  
0        {'answer_start': [235], 'text': ['하원']}               False  
1        {'answer_start': [235], 'text': ['하원']}                True  
2  {'

In [42]:
# Dataset으로 변환 (from pandas DataFrame)
train_dataset_augmented = Dataset.from_pandas(augmented_df)

# 동일하게 validation과 test도 처리 가능
valid_dataset_augmented = Dataset.from_pandas(valid_df)

# DatasetDict로 결합
dataset_dict = DatasetDict({
    "train": train_dataset_augmented,
    "validation": valid_dataset_augmented,
})

# Arrow 파일로 저장
output_dir = os.path.join(parent_dir, 'data', 'augmented_dataset_syno')
dataset_dict.save_to_disk(output_dir)

# 확인
print(f"Augmented dataset saved to {output_dir}")

Saving the dataset (1/1 shards): 100%|██████████| 240/240 [00:00<00:00, 59553.51 examples/s]

Augmented dataset saved to D:\boost-camp\07주_MRC\pj\level2-mrc-nlp-04\data\augmented_dataset_syno
